### This notebook contains an example code showing how to slew to 5 minutes before transit, wait 5 minutes after transit and then slew back to 5 minutes before transit.
# This is for daytime testing of the nasmyth rotator AND HAS NOT BEED TESTED

In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

from lsst.ts import salobj

from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [2]:
# Make IDLs separately
# make_idl_files.py ATAOS ATPneumatics ATDomeTrajectory ATHexapod

d = salobj.Domain()

atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=None)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

await asyncio.gather(atmcs.start_task,
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

RuntimeError: Failed to create topic ATDome.abort

In [7]:
d = salobj.Domain()

atdome = salobj.Remote(d, "ATDome", index=2)
await asyncio.gather(atdome.start_task)

RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 70 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages


RuntimeError: Failed to create topic ATDome.abort

In [3]:
await salobj.set_summary_state(atdome, salobj.State.STANDBY, settingsToApply='Default1')

NameError: name 'atdome' is not defined

In [6]:
# Get summary state
print(salobj.State(athex.evt_summaryState.get().summaryState))

State.STANDBY


In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
nruns = 5
​
for i in range(nruns):
    # Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
    print(f"Run {i+1} of {nruns}...")
    
    time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=5.)
    lst = Angle(time_data.lst, unit=u.hour)
    delta = Angle(10., unit=u.arcminute)
    ra = lst + delta
    print("=======================")
    print(f"LST: {lst}")
    print(f"delta: {delta}")
    print(f"RA: {ra}")
    print("=======================")
    
    # Next we get declination to be 10 degrees south of zenith. 
    dec = location.lat - Angle(6., unit=u.deg)
    print(f"Dec: {dec}")
    
    target_name="Slew to Transit test"
    radec = ICRS(ra, dec)
    
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
    coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
    alt_az = radec.transform_to(coord_frame_altaz)
    
    await atptg.cmd_raDecTarget.set_start(
        targetName=target_name,
        targetInstance=ATPtg.TargetInstances.CURRENT,
        frame=ATPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=radec.ra.hour,
        declination=radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        rotPA=180.-alt_az.alt.deg,
        rotFrame=ATPtg.RotFrame.FIXED,
        rotMode=ATPtg.RotMode.FIELD,
        timeout=10
    )
    
    print("Sleep for 2.5 minutes")
    
    await asyncio.sleep(2.5*60.)

await atptg.cmd_stopTracking.start(timeout=30)
#  